# esstimate node health
In order to find out health of a node, we need to train a model that can look at recent jobs on a node and give out a statistic

We want to train on jobs from nodes that failed right before the failure, and those that are healthy. good metric is flux. Maybe time_oncpu over cpu_time. Also start time.

In [1]:
# import  epmt query 
print('importing epmt_query')
import epmt_query as eq
# import matplot for better plotting functions
import sys
sys.path.insert(0,'/home/Avery.Kiihne/pip_experiment')
import matplotlib.pyplot as plt
import numpy as np
# import pandas. optional but helpful 'display.max_columns' arg shows all DataFrame columns when printing
print('importing pandas')
import pandas
pandas.set_option('display.max_columns', None)

importing epmt_query
importing pandas


In [2]:
#grouping nodes by hardware
# https://wiki.gfdl.noaa.gov/index.php/Post-processing_and_Analysis_User_Guide
group_1 = ['pp013', 'pp014', 'pp015', 'pp016', 'pp017',  'pp018', 'pp019', 'pp020', 'pp021', 'pp022', 'pp023', 'pp024', 'pp025', 'pp026', 'pp027', 'pp028', 'pp029', 'pp030', 'pp031', 'pp032', 'pp033', 'pp034', 'pp035', 'pp036', 'pp037', 'pp038', 'pp039', 'pp040', 'pp041', 'pp042', 'pp043', 'pp044', 'pp045', 'pp046', 'pp047', 'pp048', 'pp049', 'pp050', 'pp051', 'pp052', 'pp053', 'pp054', 'pp055', 'pp056', 'pp057', 'pp058', 'pp059', 'pp060', 'pp061', 'pp062', 'pp063', 'pp064', 'pp065', 'pp066', 'pp068', 'pp071', 'pp072', 'pp073', 'pp074', 'pp075']
group_2 = ['pp101', 'pp102', 'pp103', 'pp104', 'pp105', 'pp201', 'pp202', 'pp200']
group_3 = ['pp203', 'pp204', 'pp205', 'pp206', 'pp207', 'pp208', 'pp209', 'pp210', 'pp211', 'pp212']
group_4 = ['pp300', 'pp301', 'pp302', 'pp303', 'pp304', 'pp305', 'pp306', 'pp307', 'pp308', 'pp309', 'pp311', 'pp312', 'pp313', 'pp314', 'pp315', 'pp316', 'pp317', 'pp318', 'pp319', 'pp320', 'pp321', 'pp322', 'pp323', 'pp324', 'pp325', 'pp326', 'pp327', 'pp328', 'pp329', 'pp330', 'pp331', 'pp332', 'pp333']
group_5 = ['pp334', 'pp335', 'pp336', 'pp337', 'pp338']
test_group = ['pp038','pp039']
#choose which group to build model with
chosen_group = group_2

In [3]:
#pull jobs from a specific node
node_jobs = []
loops = 100   #number of data matrixs retrieved
num_jobs = 10
for node_name in chosen_group:
    for aa in range(loops):
        node_jobs.append(eq.get_jobs(limit = num_jobs, before = -7, hosts = [node_name] , offset = 2*num_jobs*aa))#, tags = {'env_dict:HOST:pp038'}) #, tags = {'HOST':node_name})

In [6]:
#we want to make predictions based on node health, so we want to minimize differences due to jobs just being different
#to do this, we will use a number of created metrics that reflect different rates in the data
flux = []   #write_bytes / CPU_time   This is an aproximate for bytes per period of time. High is good.
proc_time = []   #num_procs/system_time   Average time to complete one proc. Low is good.
fraction_waiting = []   #time_waiting/time_oncpu   Fraction of the total time that is spent waiting to run. Low is good.
write_failed = []    #cancelled_write_bytes / write_bytes   Fraction of write bytes that failed. more write bytes should be more cancelled. high is bad.
rssmax = []   #baseline, currently not in use
data_frame = []
data = []
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n'] #random labels used to group
for aa in range(loops*len(chosen_group)):
    forced_relation = -1*aa#alphabet[aa]   #becuase .fit does not view data together, which can force time groups together by making them share a random variable.
    data_frame.append([])
    for job in node_jobs[aa]:

        if job.get('rssmax'):
            flux.append(job['write_bytes']/job['cpu_time'])
            proc_time.append(job['num_procs']/job['systemtime'])
            fraction_waiting.append(job['time_waiting']/job['time_oncpu'])
            write_failed.append(job['cancelled_write_bytes']/job['write_bytes'])
            rssmax.append(job['rssmax'])
            data = (flux[-1],proc_time[-1],fraction_waiting[-1],write_failed[-1])  #, forced_relation)
            data_frame[aa].extend(data)
data_frame = [ele for ele in data_frame if ele != []]   #eliminates empty sets
data_frame = [ele for ele in data_frame if len(ele) == 4*num_jobs]   #eliminates groups that do not have enough jobs

# Predictions - unsupervised
Number of imperfect ways to do this. Assign every good job in good node a score of 1, and oppsite for bad nodes. Or give massive array for each single value of good/bad.

We will start with unsupervised learning

In [7]:
from sklearn import cluster, datasets
k_means = cluster.KMeans(n_clusters=2)
k_means.fit(data_frame)
print(k_means.labels_[:])

[0 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 1 0 0 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1
 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0
 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 0 0 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 1 0 1 1 1 1 1 1 1 1 0 1 

In [8]:
from sklearn.svm import SVC 


# Supervised training
once we have a list of job ids for bad nodes, we can train a model with test data


In [33]:
#find data and give it a health classifier
#pull jobs from a specific node
curated_jobs = []
loops = 100   #number of data matrixs retrieved
num_jobs = 10
good_names = ['pp039', 'pp040', 'pp041', 'pp042', 'pp043']
bad_names = ['pp013']
for node_name in good_names:
    for aa in range(loops):
        curated_jobs.append(eq.get_jobs(limit = num_jobs, before = -7, hosts = [node_name] , offset = 2*num_jobs*aa))
for aa in range(10):
    curated_jobs.append(eq.get_jobs(limit = num_jobs, before = -21, hosts = [bad_names[0]] , offset = num_jobs*aa))

In [19]:
#we want to make predictions based on node health, so we want to minimize differences due to jobs just being different
#to do this, we will use a number of created metrics that reflect different rates in the data
flux = []   #write_bytes / CPU_time   This is an aproximate for bytes per period of time. High is good.
proc_time = []   #num_procs/system_time   Average time to complete one proc. Low is good.
fraction_waiting = []   #time_waiting/time_oncpu   Fraction of the total time that is spent waiting to run. Low is good.
write_failed = []    #cancelled_write_bytes / write_bytes   Fraction of write bytes that failed. more write bytes should be more cancelled. high is bad.
rssmax = []   #baseline, currently not in use
data_frame = []
data = []
health_stat = []
for aa in range(len(curated_jobs)):
    data_frame.append([])
    for job in curated_jobs[aa]:
        if job.get('rssmax'):
            flux.append(job['write_bytes']/job['cpu_time'])
            proc_time.append(job['num_procs']/job['systemtime'])
            fraction_waiting.append(job['time_waiting']/job['time_oncpu'])
            write_failed.append(job['cancelled_write_bytes']/job['write_bytes'])
            rssmax.append(job['rssmax'])
            data = (flux[-1],proc_time[-1],fraction_waiting[-1],write_failed[-1])  #, forced_relation)
            data_frame[aa].extend(data)
    if job['env_dict']['HOSTNAME'] in good_names and data_frame[aa] != [] and len(data_frame[aa]) == 4*num_jobs:
        health_stat.append(0)
    if job['env_dict']['HOSTNAME'] in bad_names and data_frame[aa]  != [] and len(data_frame[aa]) == 4*num_jobs:
        print('test')
        health_stat.append(1)    
data_frame = [ele for ele in data_frame if ele != []]   #eliminates empty sets
data_frame = [ele for ele in data_frame if len(ele) == 4*num_jobs]   #eliminates groups that do not have enough jobs

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC 
X_train, X_test, y_train, y_test = train_test_split( 
                        data_frame,health_stat,test_size = 0.30, random_state = 101) 
# train the model on train set without using GridSearchCV 
model = SVC() 
model.fit(X_train, y_train) 
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'gamma':['scale', 'auto'],
              'kernel': ['linear']}  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
 
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(X_test) 
   
# print classification report 
print(classification_report(y_test, grid_predictions)) 

ValueError: The number of classes has to be greater than one; got 1 class

In [14]:
y_train

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [31]:
test_jobs = eq.get_jobs(limit = num_jobs, before = -7, hosts = 'pp038' , offset = num_jobs*aa)

In [32]:
test_jobs

[]

In [12]:
node_jobs[-1]

[{'updated_at': datetime.datetime(2024, 2, 29, 13, 53, 35, 55143),
  'env_dict': {'ENV': '/usr/local/Modules/5.1.1/init/profile.sh',
   'PWD': '/home/Dana.Singh',
   'HOME': '/home/Dana.Singh',
   'LANG': 'en_US',
   'MAIL': '/var/spool/mail/Dana.Singh',
   'PATH': '/home/fms/local/opt/../epmt/4.9.2-centos-7/epmt-install/epmt:/usr/local/Modules/5.1.1/bin:/home/gfdl/an+pp/bin:/app/slurm/default/bin:/bin:/usr/bin',
   'USER': 'Dana.Singh',
   'QTDIR': '/usr/lib64/qt-3.3',
   'QTINC': '/usr/lib64/qt-3.3/include',
   'QTLIB': '/usr/lib64/qt-3.3/lib',
   'SHELL': '/bin/bash',
   'SHLVL': '2',
   'LC_ALL': 'en_US.UTF-8',
   'TMPDIR': '/xtmp/Dana.Singh/job35991929',
   'DISPLAY': 'localhost:11.0',
   'LC_TIME': 'C',
   'LOGNAME': 'Dana.Singh',
   'MANPATH': ':/opt/puppetlabs/puppet/share/man',
   'BASH_ENV': '/usr/local/Modules/5.1.1/init/bash',
   'CYLC_UTC': 'True',
   'HISTSIZE': '1000',
   'HOSTNAME': 'pp038',
   'LESSOPEN': '||/usr/bin/lesspipe.sh %s',
   'CYLC_DEBUG': 'false',
   'MODUL

In [157]:
scores = [[0,200],[1,100],[2,500],[3,400],[4,300]]
scores = sorted(scores, key=lambda score: score[1], reverse=True)
scores

[[2, 500], [3, 400], [4, 300], [0, 200], [1, 100]]

In [160]:
dictionary = {'a':1,'b':8,'c':3}
sorted(dictionary)

['a', 'b', 'c']

In [8]:
x,y = [],[]

In [37]:
y = 'a'
y in ['c','b']

False